In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
river = "Anabar"
river2 = river.replace(" ", "_")

path_data0 = os.getcwd() + "\\"
path_data1 = path_data0 + r"ERA5_Land\\"
path_data2 = path_data0 + r"Data\\"
path_df = r"Table\\"

In [ ]:
var_list1 = [
    "dewpoint_temperature_2m",
    "temperature_2m",
    "soil_temperature_level_1",
    "snow_depth_water_equivalent",
    "snowfall_sum",
    "snowmelt_sum",
    "sub_surface_runoff_sum",
    "surface_runoff_sum",
    "surface_net_solar_radiation_sum",
    "surface_net_thermal_radiation_sum",
    "total_evaporation_sum",
    "total_precipitation_sum",
    "u_component_of_wind_10m",
    "v_component_of_wind_10m",
]

var_list2 = [
    "Dewpoint_temperature_2m",
    "Temperature_2m",
    "Soil_temperature_1",
    "Snow_depth_water_equivalent",
    "Snowfall",
    "Snowmelt",
    "Sub_surface_runoff",
    "Surface_runoff",
    "Surface_net_solar_radiation",
    "Surface_net_thermal_radiation",
    "Evaporation",
    "Precipitation",
    "Wind_U",
    "Wind_V",
]

prefix_list = ["p1", "basin"]

dem_list = ["elevation", "slope", "aspect"]
dem_list2 = [f"{p}_{d}" for p in prefix_list for d in dem_list]

In [ ]:
window_sizes = [2, 15, 30, 45]  # Number of days to calculate the moving average

date1 = pd.Timestamp(1950, 1, 1)
date2 = pd.Timestamp(2023, 12, 31)
time_index = pd.date_range(start=date1, end=date2)

date_missing = pd.Timestamp(1950, 1, 1)

In [ ]:
def preprocess_era5(df, prefix):
    df[f"{prefix}_Wind"] = np.sqrt(
        df[f"{prefix}_Wind_U"] ** 2 + df[f"{prefix}_Wind_V"] ** 2
    )
    df.drop(columns=[f"{prefix}_Wind_U", f"{prefix}_Wind_V"], inplace=True)

    # fill the missing values for Dec 31 with backward fill
    d1 = df.index.min()
    d2 = df.index.max()
    time_index0 = pd.date_range(start=d1, end=d2, freq="D")
    df = df.reindex(time_index0, method="ffill")

    return df


def encode_date_to_sin_cos(dates):
    if not isinstance(dates, pd.Series):
        raise ValueError("输入必须是pandas Series")

    dates = pd.to_datetime(dates)

    is_leap_year = dates.dt.is_leap_year
    day_of_year = dates.dt.dayofyear
    days_in_year = np.where(is_leap_year, 366, 365)

    JD_sin = np.sin(day_of_year * 2 * np.pi / days_in_year)
    JD_cos = np.cos(day_of_year * 2 * np.pi / days_in_year)

    return pd.Series(JD_cos)

In [ ]:
file_station = "Station Info5.csv"
df_station = pd.read_csv(file_station)
df_station = df_station[df_station["River"] == river]
assert len(df_station) == 1
station = df_station["Station"].values[0]
station2 = station.replace(" ", "_").replace(",", "").replace(".", "").replace("'", "")

file_discharge = path_data2 + f"{river}__{station}.csv"
df2 = pd.read_csv(file_discharge, parse_dates=["Date"], index_col="Date", usecols=["Date", "Discharge"])
df2["Discharge"] = df2["Discharge"].clip(lower=0)

df2

In [ ]:
df_temp1 = pd.read_csv(path_data1 + f"{river2}__{station2}__Point_DEM.csv").drop(columns=["system:index", ".geo"]).squeeze()
df_temp2 = pd.read_csv(path_data1 + f"{river2}__{station2}__Basin_DEM.csv").drop(columns=["system:index", ".geo"]).squeeze()
df_dem = pd.concat([df_temp1, df_temp2], axis=0)

df_temp3 = pd.read_csv(path_data1 + f"{river2}__{station2}__Point_ERA5_Land.csv", parse_dates=["date"], index_col="date")
df_temp4 = pd.read_csv(path_data1 + f"{river2}__{station2}__Basin_ERA5_Land.csv", parse_dates=["date"], index_col="date")

df1 = pd.concat([df_temp3, df_temp4], axis=1)
df1.index.names = ["Date"]
df1.drop(columns=["system:index", ".geo"], inplace=True)
if date1 <= date_missing <= date2:
    # fill the values of "1950-01-01" with "1950-01-02"
    df1.loc[date_missing] = np.nan
    df1.sort_index(inplace=True)
    df1.fillna(method="bfill", inplace=True)

for prefix in prefix_list:
    col_list1 = [f"{prefix}_{col}" for col in var_list1]
    col_list2 = [f"{prefix}_{col}" for col in var_list2]
    df1.rename(columns=dict(zip(col_list1, col_list2)), inplace=True)
    df1 = preprocess_era5(df1, prefix)

enviro_vars = df1.columns.tolist()

for dem in df_dem.index:
    df1[dem] = df_dem[dem]
    
df1["JD"] = df1.index.dayofyear
df1["JD_cos"] = encode_date_to_sin_cos(df1.index.to_series())

cols = df1.columns.tolist()
cols = cols[-2:] + cols[:-2]
df1 = df1[cols]

df1

In [ ]:
df3 = pd.concat([df1, df2], axis=1)
df3 = df3.resample("D").mean()

df3.insert(0, "River", river)
df3.insert(1, "Station", station)

df3

In [ ]:
# Rolling mean
var_list3 = df3.columns.drop(
    ["River", "Station", "Discharge", "JD", "JD_cos"] + df_dem.index.tolist()
).tolist()

for enviro_var in enviro_vars:
    for ws in window_sizes:
        df3[f"{enviro_var}__{ws}"] = df3[enviro_var].rolling(window=ws, min_periods=1).mean()

df3 = df3.copy()
df3 = df3.reindex(time_index)
df3.index.name = "Date"

# Drop not rolling mean columns
df3 = df3.drop(columns=var_list3)
print(df3.shape)
df3.to_csv(path_df + "Data0_with_DEM.csv", index=True)

In [ ]:
for col in df3.columns:
    for col2 in dem_list:
        if col2 in col:
            df3.drop(columns=col, inplace=True)

df3.to_csv(path_df + "Data1_with_nan.csv", index=True)
print(df3.shape)
df3

In [ ]:
df4 = df3.dropna(how="any")
df4.to_csv(path_df + "Data2_without_nan.csv", index=True)
print(df4.shape)

ratio = df4.shape[0] / df3.shape[0] * 100
print("Ratio of non-NaN data: {:.2f}%".format(ratio))

if df3.shape[0] == df4.shape[0]:
    print("No NaN data in the DataFrame.")